In [107]:
import pandas as pd
import mysql.connector

# Define your MySQL connection parameters
config = {
    'user': 'root',
    'password': '',
    'host': '127.0.0.1',  # e.g., 'localhost' or an IP address
    'database': 'meta-susenas'
}

# Create a connection to the database
connection = mysql.connector.connect(**config)

# Define your SQL query
query_vsusenas_mak = """
SELECT v.*, count(art.id) as jumlah_art  FROM vsusenas_mak v
LEFT JOIN anggota_ruta art ON art.id_ruta=v.id
GROUP BY v.id
"""
query_konsumsi_ruta = """
SELECT krt.*, 
v.status_dok, v.kode_prov,.v.kode_kabkot,v.nks,v.r202_nama as pml,
nama_komoditas,nama_kelompok, kd.type
FROM konsumsi krt
LEFT JOIN vsusenas_mak v ON krt.id_ruta=v.id
LEFT JOIN komoditas kd ON kd.id=krt.id_komoditas
"""
query_konsumsi_art = """
SELECT kart.*, 
art.nama as nama_art,
v.id as id_ruta, v.status_dok, v.kode_prov,.v.kode_kabkot,v.nks,v.r202_nama as pml,
nama_komoditas,nama_kelompok, kd.type
FROM konsumsi_art kart
LEFT JOIN anggota_ruta art ON kart.id_art=art.id
LEFT JOIN vsusenas_mak v ON art.id_ruta=v.id
LEFT JOIN komoditas kd ON kd.id=kart.id_komoditas
"""

# Read data into a DataFrame
vsusenas_mak = pd.read_sql(query_vsusenas_mak, connection)
konsumsi_ruta = pd.read_sql(query_konsumsi_ruta, connection)
konsumsi_art = pd.read_sql(query_konsumsi_art, connection)
# inspec_df = pd.read_sql(inspect_,connection)
# Close the connection
connection.close()


C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\2289093794.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vsusenas_mak = pd.read_sql(query_vsusenas_mak, connection)
C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\2289093794.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  konsumsi_ruta = pd.read_sql(query_konsumsi_ruta, connection)
C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\2289093794.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  konsumsi_art = pd.read_sql(query_konsumsi_art, connection)


In [108]:
# konsumsi_ruta = konsumsi_ruta.loc[:, ~konsumsi_ruta.columns.duplicated()]
konsumsi_art.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206349 entries, 0 to 206348
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               206349 non-null  int64  
 1   id_komoditas     206349 non-null  int64  
 2   id_art           206349 non-null  object 
 3   item             206349 non-null  object 
 4   satuan           206349 non-null  object 
 5   harga_beli       206349 non-null  int64  
 6   harga_produksi   206349 non-null  int64  
 7   volume_beli      206349 non-null  float64
 8   volume_produksi  206349 non-null  float64
 9   volume_total     206349 non-null  float64
 10  harga_total      206349 non-null  int64  
 11  nama_art         206310 non-null  object 
 12  id_ruta          205101 non-null  object 
 13  status_dok       205101 non-null  object 
 14  kode_prov        205101 non-null  object 
 15  kode_kabkot      205101 non-null  object 
 16  nks              205101 non-null  obje

In [109]:
warning_columns = ["id_ruta","status_dok","kode_prov","kode_kabkot","nks","rincian","blok","warning"]
warning_ruta = pd.DataFrame(columns=warning_columns)

# warning_konsumsi_ruta = ["id_ruta","status_dok","kode_prov","kode_kabkot","nks","rincian","blok","warning"]
def validasi_konsumsi(konsumsi,common_warning,blok):
    common_warning["rincian"]=f"Kode Komoditas [{konsumsi["id_komoditas"]}] {konsumsi["nama_komoditas"]}"
    if konsumsi["id_komoditas"]==159 and konsumsi["harga_beli"]==0 and konsumsi["harga_produksi"]==0:
        common_warning["blok"]=blok
        common_warning["warning"]="Jumlah Konsumsi ART Makanan dan Minuman (Komoditas 159) tidak boleh kosong"
        warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    if konsumsi["harga_total"] != konsumsi["harga_beli"] + konsumsi["harga_produksi"]:
        common_warning["blok"]=blok
        common_warning["warning"]="Harga total tidak sama dengan penjumlahan harga beli dan harga produksi"
        warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    if konsumsi["type"]!="sub":
        if konsumsi["volume_total"]!= konsumsi["volume_beli"]+konsumsi["volume_produksi"]:
            common_warning["blok"]=blok
            common_warning["warning"]="Volume total tidak sama dengan penjumlahan volume beli dan volume produksi"
            warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
        if((konsumsi["harga_produksi"]>0 and konsumsi["volume_produksi"]==0) or (konsumsi["harga_produksi"]==0 and konsumsi["volume_produksi"]>0))  :
            common_warning["blok"]=blok
            common_warning["warning"]="Volume dan Harga untuk Produksi tidak sejalan (Salah satu bernilai 0)"
            warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
        if ((konsumsi["harga_produksi"]>0 and konsumsi["volume_produksi"]==0) or (konsumsi["harga_produksi"]==0 and konsumsi["volume_produksi"]>0))  :
            common_warning["blok"]=blok
            common_warning["warning"]="Volume dan Harga untuk Produksi tidak sejalan (Salah satu bernilai 0)"
            warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    # if (konsumsi["harga_produksi"])    
def validasi_konsumsi_art(konsumsi):
    ruta = vsusenas_mak[vsusenas_mak["id"]==konsumsi["id_ruta"]]
         
    if ruta.empty:
        return
    ruta=ruta.iloc[0]
    common_warning = {
    "id_ruta":ruta[0],
    "status_dok":ruta[2],
    "kode_prov":ruta[3],
    "kode_kabkot":ruta[4],
    "nks":ruta[8],
    "pml":ruta["r202_nama"]
}
    blok=f"Konsumsi ART nama {konsumsi["nama_art"]}"
    validasi_konsumsi(konsumsi,common_warning,blok)

    
def validasi_ruta(ruta):
   
    # print("ruta : ",ruta)
    # return
    common_warning = {
    "id_ruta":ruta[0],
    "status_dok":ruta[2],
    "kode_prov":ruta[3],
    "kode_kabkot":ruta[4],
    "nks":ruta[8],
    "pml":ruta["r202_nama"]
}
    
    # cek wtf 2 (jumlah art)
    if(not bool(ruta["wtf_1"])):
        #add
        common_warning["rincian"]="1. Jumlah ART"
        common_warning["blok"]="Worksheet"
        common_warning["warning"]="Jumlah ART tidak boleh kosong"
        warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    elif ruta["wtf_1"]<1:
        common_warning["rincian"]="1. Jumlah ART"
        common_warning["blok"]="Worksheet"
        common_warning["warning"]="Jumlah ART minimal 1"
        warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    
    # cek wtf 2 (jumlah balita)
    if ruta["wtf_2"]>=ruta["wtf_1"]:
        common_warning["rincian"]="2. Jumlah Balita"
        common_warning["blok"]="Worksheet"
        common_warning["warning"]="Jumlah Balita tidak boleh sama atau lebih dari Jumlah ART "
        warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    # cek wtf_3 atau jumlah art yang bersekolah
    if ruta["wtf_11"]>ruta["wtf_3"]:
        common_warning["rincian"]="11. Jumlah ART yang menerima PIP"
        common_warning["blok"]="Worksheet"
        common_warning["warning"]="Jumlah ART yang menerima PIP tidak boleh lebih dari Jumlah ART yang masih bersekolah"
        warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    if ruta["wtf_1"]!=ruta["jumlah_art"]:
        common_warning["rincian"]="Jumlah ART Worksheet dan Blok IV"
        common_warning["blok"]="Worksheet dan Blok IV"
        common_warning["warning"]="Jumlah ART yang pada Blok IV dan pada Blok Worksheet tidak sama"
        warning_ruta.loc[len(warning_ruta)] = common_warning # Append the new row at the end
    if int(ruta['r203'])<3:
        # periksa konsumsi ruta
        blok="Konsumsi Ruta"
        filtered_konsumsi_ruta = konsumsi_ruta[konsumsi_ruta["id_ruta"]==ruta["id"]]
        filtered_konsumsi_ruta.apply(validasi_konsumsi,args=(common_warning,blok),axis=1)


        
    # return warning_ruta

# Warning RUTA

In [110]:
vsusenas_mak.apply(validasi_ruta,axis=1)

C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\202422066.py:52: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "id_ruta":ruta[0],
C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\202422066.py:53: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "status_dok":ruta[2],
C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\202422066.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "kode_prov":ruta[3],
C:\Users\Ponimin\AppData\Local\T

0       None
1       None
2       None
3       None
4       None
        ... 
1722    None
1723    None
1724    None
1725    None
1726    None
Length: 1727, dtype: object

# Warning ART

In [111]:
konsumsi_art.apply(validasi_konsumsi_art,axis=1)

C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\202422066.py:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "id_ruta":ruta[0],
C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\202422066.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "status_dok":ruta[2],
C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\202422066.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  "kode_prov":ruta[3],
C:\Users\Ponimin\AppData\Local\T

0         None
1         None
2         None
3         None
4         None
          ... 
206344    None
206345    None
206346    None
206347    None
206348    None
Length: 206349, dtype: object

In [119]:
vsusenas_mak.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 97 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              1727 non-null   object        
 1   users_id                        1727 non-null   object        
 2   status_dok                      1727 non-null   object        
 3   kode_prov                       1727 non-null   object        
 4   kode_kabkot                     1727 non-null   object        
 5   kode_kec                        1727 non-null   object        
 6   kode_desa                       1727 non-null   object        
 7   kode_bs4                        1727 non-null   object        
 8   nks                             1727 non-null   object        
 9   semester                        1727 non-null   object        
 10  r108                            1727 non-null   object        
 11  r109

In [122]:
warning_ruta["pml"]="Nama PML"
warning_ruta["kepala_ruta"]="Nama Kepala RUTA"

def add_variables(row):
    ruta = vsusenas_mak[vsusenas_mak["id"]==row["id_ruta"]]
    if not ruta.empty:
        ruta=ruta.iloc[0]
        row["pml"]=ruta["r202_nama"]
        row["kepala_ruta"]=ruta["r110"]
    return row


In [124]:
warning_ruta_update = warning_ruta.apply(add_variables,axis=1)

In [130]:
warning_art = warning_ruta_update[warning_ruta_update["blok"].str.contains("art",case=False)]
warning_ruta = warning_ruta_update[~warning_ruta_update["blok"].str.contains("art",case=False)]
warning_art

,id_ruta,status_dok,kode_prov,kode_kabkot,nks,rincian,blok,warning,pml,kepala_ruta
2982,9d136a08-1bb8-4701-8cb2-be643782b15a,clean,71,72,250051,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama ALIF,Harga total tidak sama dengan penjumlahan harg...,Johanna MF Tampemawa,HUSAIN ARSYAO
2983,9d137c94-bc11-4655-9a62-7411cee3d65d,clean,71,02,200295,"Kode Komoditas [163] Kue basah (kue lapis, bik...",Konsumsi ART nama JUSOP RENGKUNG,Volume total tidak sama dengan penjumlahan vol...,Novi Yanti Mamangkey,JUSOP RENGKU
2984,9d137c94-bc11-4655-9a62-7411cee3d65d,clean,71,02,200295,Kode Komoditas [181] Bubur ayam,Konsumsi ART nama SOPIA KAWUWUNG,Volume total tidak sama dengan penjumlahan vol...,Novi Yanti Mamangkey,JUSOP RENGKU
2985,9d154178-0092-44b8-9246-2cd064b566d7,clean,71,02,200453,"Kode Komoditas [162] Kue kering, biskuit, semp...",Konsumsi ART nama FREDIKA KANDOUW,Harga total tidak sama dengan penjumlahan harg...,Tirsa Poli,Fredika Kandouw
2986,9d154178-0092-44b8-9246-2cd064b566d7,clean,71,02,200453,"Kode Komoditas [167] Gado-gado, ketoprak, pecel",Konsumsi ART nama FREDIKA KANDOUW,Harga total tidak sama dengan penjumlahan harg...,Tirsa Poli,Fredika Kandouw
...,...,...,...,...,...,...,...,...,...,...
3746,9d27c610-0c72-48f7-a442-3d70af60ece4,warning,71,01,200078,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama New,Harga total tidak sama dengan penjumlahan harg...,Rani Adila,VERNI TINDAGE
3747,9d292c95-ea7d-40e8-8a5f-ffa948e61674,clean,71,74,250147,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama heny,Harga total tidak sama dengan penjumlahan harg...,Angel Yeane Lombogia,Sunandar Latief
3748,9d30369f-a653-41dd-babf-e3b959a1de49,clean,71,11,200173,"Kode Komoditas [161] Roti manis, roti lainnya",Konsumsi ART nama Nouel,Harga total tidak sama dengan penjumlahan harg...,Muchlis Manoppo,Fentje Mumek
3749,9d30369f-a653-41dd-babf-e3b959a1de49,clean,71,11,200173,"Kode Komoditas [161] Roti manis, roti lainnya",Konsumsi ART nama Nouel,Volume total tidak sama dengan penjumlahan vol...,Muchlis Manoppo,Fentje Mumek


In [131]:
warning_ruta

,id_ruta,status_dok,kode_prov,kode_kabkot,nks,rincian,blok,warning,pml,kepala_ruta
0,9d1351e7-7263-4fe3-a338-027341c7f61d,clean,71,06,200029,Kode Komoditas [75] F. SAYUR-SAYURAN [R.76 s.d...,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Usman Antu,STENLY WUISAN
1,9d1351e7-7263-4fe3-a338-027341c7f61d,clean,71,06,200029,Kode Komoditas [110] H. BUAH-BUAHAN [R.111 s.d...,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Usman Antu,STENLY WUISAN
2,9d13522e-81dc-4b55-b133-aa99382359a2,clean,71,06,200029,Kode Komoditas [75] F. SAYUR-SAYURAN [R.76 s.d...,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Usman Antu,ALFRITS MALONDA
3,9d1352a4-a475-456a-9fda-802e4b1d1e11,clean,71,06,200029,Kode Komoditas [75] F. SAYUR-SAYURAN [R.76 s.d...,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Usman Antu,YORRI ROMPIS
4,9d1367f8-61f3-4999-b502-7746bef68fdf,clean,71,06,200029,Kode Komoditas [65] E. TELUR DAN SUSU [R.66 s....,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Usman Antu,MEIKE ROTTY
...,...,...,...,...,...,...,...,...,...,...
2977,9d30078b-cbe9-408a-90a0-24aede7f244d,clean,71,11,200195,Kode Komoditas [18] Tongkol,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Muchlis Manoppo,Toni Rumondor
2978,9d30078b-cbe9-408a-90a0-24aede7f244d,clean,71,11,200195,"Kode Komoditas [20] Cakalang, dencis",Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Muchlis Manoppo,Toni Rumondor
2979,9d30078b-cbe9-408a-90a0-24aede7f244d,clean,71,11,200195,Kode Komoditas [66] Telur ayam ras,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Muchlis Manoppo,Toni Rumondor
2980,9d30078b-cbe9-408a-90a0-24aede7f244d,clean,71,11,200195,Kode Komoditas [118] Pisang ambon,Konsumsi Ruta,Harga total tidak sama dengan penjumlahan harg...,Muchlis Manoppo,Toni Rumondor


In [134]:
warning_art[["blok_","nama_art"]] = warning_art["blok"].str.split(" nama ",expand=True)
warning_art.drop(columns=["blok"])
warning_art

C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\3070094871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  warning_art[["blok_","nama_art"]] = warning_art["blok"].str.split(" nama ",expand=True)
C:\Users\Ponimin\AppData\Local\Temp\ipykernel_8728\3070094871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  warning_art[["blok_","nama_art"]] = warning_art["blok"].str.split(" nama ",expand=True)


,id_ruta,status_dok,kode_prov,kode_kabkot,nks,rincian,blok,warning,pml,kepala_ruta,blok_,nama_art
2982,9d136a08-1bb8-4701-8cb2-be643782b15a,clean,71,72,250051,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama ALIF,Harga total tidak sama dengan penjumlahan harg...,Johanna MF Tampemawa,HUSAIN ARSYAO,Konsumsi ART,ALIF
2983,9d137c94-bc11-4655-9a62-7411cee3d65d,clean,71,02,200295,"Kode Komoditas [163] Kue basah (kue lapis, bik...",Konsumsi ART nama JUSOP RENGKUNG,Volume total tidak sama dengan penjumlahan vol...,Novi Yanti Mamangkey,JUSOP RENGKU,Konsumsi ART,JUSOP RENGKUNG
2984,9d137c94-bc11-4655-9a62-7411cee3d65d,clean,71,02,200295,Kode Komoditas [181] Bubur ayam,Konsumsi ART nama SOPIA KAWUWUNG,Volume total tidak sama dengan penjumlahan vol...,Novi Yanti Mamangkey,JUSOP RENGKU,Konsumsi ART,SOPIA KAWUWUNG
2985,9d154178-0092-44b8-9246-2cd064b566d7,clean,71,02,200453,"Kode Komoditas [162] Kue kering, biskuit, semp...",Konsumsi ART nama FREDIKA KANDOUW,Harga total tidak sama dengan penjumlahan harg...,Tirsa Poli,Fredika Kandouw,Konsumsi ART,FREDIKA KANDOUW
2986,9d154178-0092-44b8-9246-2cd064b566d7,clean,71,02,200453,"Kode Komoditas [167] Gado-gado, ketoprak, pecel",Konsumsi ART nama FREDIKA KANDOUW,Harga total tidak sama dengan penjumlahan harg...,Tirsa Poli,Fredika Kandouw,Konsumsi ART,FREDIKA KANDOUW
...,...,...,...,...,...,...,...,...,...,...,...,...
3746,9d27c610-0c72-48f7-a442-3d70af60ece4,warning,71,01,200078,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama New,Harga total tidak sama dengan penjumlahan harg...,Rani Adila,VERNI TINDAGE,Konsumsi ART,New
3747,9d292c95-ea7d-40e8-8a5f-ffa948e61674,clean,71,74,250147,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama heny,Harga total tidak sama dengan penjumlahan harg...,Angel Yeane Lombogia,Sunandar Latief,Konsumsi ART,heny
3748,9d30369f-a653-41dd-babf-e3b959a1de49,clean,71,11,200173,"Kode Komoditas [161] Roti manis, roti lainnya",Konsumsi ART nama Nouel,Harga total tidak sama dengan penjumlahan harg...,Muchlis Manoppo,Fentje Mumek,Konsumsi ART,Nouel
3749,9d30369f-a653-41dd-babf-e3b959a1de49,clean,71,11,200173,"Kode Komoditas [161] Roti manis, roti lainnya",Konsumsi ART nama Nouel,Volume total tidak sama dengan penjumlahan vol...,Muchlis Manoppo,Fentje Mumek,Konsumsi ART,Nouel


In [135]:
warning_art

,id_ruta,status_dok,kode_prov,kode_kabkot,nks,rincian,blok,warning,pml,kepala_ruta,blok_,nama_art
2982,9d136a08-1bb8-4701-8cb2-be643782b15a,clean,71,72,250051,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama ALIF,Harga total tidak sama dengan penjumlahan harg...,Johanna MF Tampemawa,HUSAIN ARSYAO,Konsumsi ART,ALIF
2983,9d137c94-bc11-4655-9a62-7411cee3d65d,clean,71,02,200295,"Kode Komoditas [163] Kue basah (kue lapis, bik...",Konsumsi ART nama JUSOP RENGKUNG,Volume total tidak sama dengan penjumlahan vol...,Novi Yanti Mamangkey,JUSOP RENGKU,Konsumsi ART,JUSOP RENGKUNG
2984,9d137c94-bc11-4655-9a62-7411cee3d65d,clean,71,02,200295,Kode Komoditas [181] Bubur ayam,Konsumsi ART nama SOPIA KAWUWUNG,Volume total tidak sama dengan penjumlahan vol...,Novi Yanti Mamangkey,JUSOP RENGKU,Konsumsi ART,SOPIA KAWUWUNG
2985,9d154178-0092-44b8-9246-2cd064b566d7,clean,71,02,200453,"Kode Komoditas [162] Kue kering, biskuit, semp...",Konsumsi ART nama FREDIKA KANDOUW,Harga total tidak sama dengan penjumlahan harg...,Tirsa Poli,Fredika Kandouw,Konsumsi ART,FREDIKA KANDOUW
2986,9d154178-0092-44b8-9246-2cd064b566d7,clean,71,02,200453,"Kode Komoditas [167] Gado-gado, ketoprak, pecel",Konsumsi ART nama FREDIKA KANDOUW,Harga total tidak sama dengan penjumlahan harg...,Tirsa Poli,Fredika Kandouw,Konsumsi ART,FREDIKA KANDOUW
...,...,...,...,...,...,...,...,...,...,...,...,...
3746,9d27c610-0c72-48f7-a442-3d70af60ece4,warning,71,01,200078,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama New,Harga total tidak sama dengan penjumlahan harg...,Rani Adila,VERNI TINDAGE,Konsumsi ART,New
3747,9d292c95-ea7d-40e8-8a5f-ffa948e61674,clean,71,74,250147,Kode Komoditas [159] M. MAKANAN DAN MINUMAN JA...,Konsumsi ART nama heny,Harga total tidak sama dengan penjumlahan harg...,Angel Yeane Lombogia,Sunandar Latief,Konsumsi ART,heny
3748,9d30369f-a653-41dd-babf-e3b959a1de49,clean,71,11,200173,"Kode Komoditas [161] Roti manis, roti lainnya",Konsumsi ART nama Nouel,Harga total tidak sama dengan penjumlahan harg...,Muchlis Manoppo,Fentje Mumek,Konsumsi ART,Nouel
3749,9d30369f-a653-41dd-babf-e3b959a1de49,clean,71,11,200173,"Kode Komoditas [161] Roti manis, roti lainnya",Konsumsi ART nama Nouel,Volume total tidak sama dengan penjumlahan vol...,Muchlis Manoppo,Fentje Mumek,Konsumsi ART,Nouel


In [138]:

# Use ExcelWriter to write multiple sheets
with pd.ExcelWriter("warning.xlsx", engine='openpyxl', mode='a') as writer:
    warning_ruta.to_excel(writer, sheet_name="RUTA", index=False)
    warning_art.to_excel(writer, sheet_name="ART", index=False)